In [3]:
#OPT 模型（Open Pre-trained Transformer）是 Meta 开源的大型语言模型，其核心是 Next Token Prediction（下一个词预测）任务。
#作用：
#  根据上下文预测下一个最可能的词/标记（token）
#  用于文本生成（续写故事、对话、代码等）
#  学习语言统计规律和语义关系
#Next Token Prediction 原理：
#  输入文本被拆分为 tokens（如："Hello" → ["He", "llo"]）
#  模型通过自注意力机制计算上下文表示
#  输出层预测下一个 token 的概率分布
#  选择概率最高的 token（或按概率采样）作为预测结果

#export HF_ENDPOINT=https://hf-mirror.com
#huggingface-cli download facebook/opt-125m --local-dir /mnt/workspace/ai/models/opt-125m

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# 1. 加载 OPT 模型和分词器（使用小规模版本节省资源）
model_name = "/mnt/workspace/ai/models/opt-125m"  # 可选：opt-350m, opt-1.3b 等
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


In [6]:

# 2. 输入文本
text = "The future of artificial intelligence"

# 3. 编码输入 → 生成 token IDs
input_ids = tokenizer.encode(text, return_tensors="pt")  # 格式: torch tensor

# 4. 预测下一个 token (贪婪搜索)
with torch.no_grad():  #推理禁用梯度
    outputs = model(input_ids)
    next_token_logits = outputs.logits[:, -1, :]  # 取最后一个位置的 logits
    
# 5. 计算概率分布
probs = torch.softmax(next_token_logits, dim=-1) #激活函数
topk_probs, topk_tokens = torch.topk(probs, 5, dim=-1)  # 取概率最高的 5 个 token

# 6. 解码并打印结果
print(f"输入文本: '{text}'\n")
print("预测的下一个 token 及概率:")
for i in range(5):
    token = tokenizer.decode(topk_tokens[0][i])
    prob = topk_probs[0][i].item()
    print(f"{i+1}. {token:<10} | 概率: {prob:.4f}")

# 7. 完整文本生成（续写 20 个 token）
generated = model.generate(
    input_ids,
    max_length=len(input_ids[0]) + 20,  # 续写长度
    do_sample=True,          # 启用采样（非贪婪）
    top_k=50,                # 从高概率的 50 个 token 中采样
    temperature=0.7,         # 控制随机性（0=确定，1=随机）
)
print("\n生成完整续写:")
print(tokenizer.decode(generated[0], skip_special_tokens=True))

输入文本: 'The future of artificial intelligence'

预测的下一个 token 及概率:
1.  is        | 概率: 0.2814
2. 
          | 概率: 0.1943
3.  and       | 概率: 0.0496
4.  (         | 概率: 0.0469
5.  will      | 概率: 0.0403

生成完整续写:
The future of artificial intelligence

The future of artificial intelligence is a subject that is constantly on the mind of many many people
